In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from sklearn.preprocessing import MultiLabelBinarizer

# Initialize BigQuery client
client = bigquery.Client()


In [2]:
# Define your query
query = """
SELECT *
FROM `GDELT.actor_event_covid`
"""

# Execute the query and load the data into a pandas DataFrame
df = client.query(query).to_dataframe()

/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
I0000 00:00:1724294936.300549   47674 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listen

In [3]:
df.head()

,GLOBALEVENTID,SQLDATE,Actor1Name,Actor2Name,EventRootCode,EventBaseCode,EventCode,NumMentions,SOURCEURL
0,1041690933,20220428,MEDIA,UNITED STATES,10,104,1043,7,http://www.msn.com/en-us/news/us/taylor-lorenz...
1,725576477,20180126,AUTHORITIES,UNITED STATES,07,073,073,10,https://www.perthnow.com.au/lifestyle/house-of...
2,484074922,20151112,WASHINGTON,COMPANY,19,192,192,20,http://pamplinmediagroup.com/pt/9-news/281005-...
3,1049258406,20220614,POLICE,WORKER,16,160,160,3,https://www.fredericknewspost.com/news/economy...
4,1093305478,20230404,UNITED STATES,AUTHORITIES,15,152,152,4,https://www.nny360.com/top_stories/nyc-braces-...


In [4]:
# Convert SQLDATE to datetime to extract month and day
df['SQLDATE'] = pd.to_datetime(df['SQLDATE'], format='%Y%m%d')

# Calculate the time feature
df['time'] = (30 * df['SQLDATE'].dt.month + df['SQLDATE'].dt.day) / 365

In [5]:
df.head()

,GLOBALEVENTID,SQLDATE,Actor1Name,Actor2Name,EventRootCode,EventBaseCode,EventCode,NumMentions,SOURCEURL,time
0,1041690933,2022-04-28,MEDIA,UNITED STATES,10,104,1043,7,http://www.msn.com/en-us/news/us/taylor-lorenz...,0.405479
1,725576477,2018-01-26,AUTHORITIES,UNITED STATES,07,073,073,10,https://www.perthnow.com.au/lifestyle/house-of...,0.153425
2,484074922,2015-11-12,WASHINGTON,COMPANY,19,192,192,20,http://pamplinmediagroup.com/pt/9-news/281005-...,0.936986
3,1049258406,2022-06-14,POLICE,WORKER,16,160,160,3,https://www.fredericknewspost.com/news/economy...,0.531507
4,1093305478,2023-04-04,UNITED STATES,AUTHORITIES,15,152,152,4,https://www.nny360.com/top_stories/nyc-braces-...,0.339726


In [6]:
# Count the frequency of each actor
actor_counts = pd.concat([df['Actor1Name'], df['Actor2Name']]).value_counts()

# Filter actors based on the minimum number of occurrences
important_actors = actor_counts.index
print("actor vector: {}".format(important_actors))

actor vector: Index(['UNITED STATES', 'SCHOOL', 'POLICE', 'PRESIDENT', 'COMMUNITY',
       'STUDENT', 'BUSINESS', 'COMPANY', 'ATTORNEY', 'JUDGE', 'AMERICAN',
       'GOVERNMENT', 'UNIVERSITY', 'SENATE', 'PRISON', 'MEDIA', 'WASHINGTON',
       'GOVERNOR', 'HOSPITAL', 'COLLEGE', 'NEW YORK', 'VOTER', 'EMPLOYEE',
       'TEXAS', 'CRIMINAL', 'RESIDENTS', 'AUTHORITIES', 'CALIFORNIA',
       'CONGRESS', 'THE WHITE HOUSE', 'FLORIDA', 'LAWMAKER', 'ADMINISTRATION',
       'SENATOR', 'WORKER', 'MAYOR', 'CHINA', 'MILITARY', 'INDUSTRY',
       'PROSECUTOR', 'PROTESTER', 'JOE BIDEN', 'AFRICA', 'LAWYER', 'OHIO',
       'UNITED KINGDOM', 'MICHIGAN', 'CHICAGO', 'SUPREME COURT', 'IOWA',
       'COMPANIES', 'VIRGINIA', 'WEBSITE', 'DOCTOR', 'THE US',
       'HEALTH OFFICIAL'],
      dtype='object')


In [7]:
len(important_actors)

56

In [ ]:
# Function to filter and multi-hot encode actors
def multi_hot_encode_actors(row, important_actors):
    actor_vector = np.zeros(len(important_actors), dtype=int)
    actor_vector[important_actors.get_loc(row['Actor1Name'])] = 1
    actor_vector[important_actors.get_loc(row['Actor2Name'])] = 1
    return actor_vector

# Apply multi-hot encoding for actors
df['actor_vector'] = df.apply(multi_hot_encode_actors, important_actors=important_actors, axis=1)


In [ ]:
df['actor_vector'].head()

In [ ]:
# Combine unique codes from all three columns
all_unique_codes = sorted(set(df['EventRootCode'].unique()) )# |
                          #set(df['EventBaseCode'].unique()) |
                          #set(df['EventCode'].unique()))

# Create a mapping of codes to indices
code_to_index = {code: idx for idx, code in enumerate(all_unique_codes)}

In [ ]:
all_unique_codes

In [ ]:
len(code_to_index)

In [ ]:
# Function to encode a single row
def encode_row(row, code_to_index):
    # Create a vector of zeros with length equal to the number of unique codes
    vector = np.zeros(len(code_to_index), dtype=int)
    
    # Set the index for each code in the vector to 1
    for col in ['EventRootCode', 'EventBaseCode', 'EventCode']:
        code = row[col]
        if code in code_to_index:
            vector[code_to_index[code]] = 1
        
    return vector

# Apply the encoding to each row in the dataframe
df['event_vector'] = df.apply(encode_row, code_to_index=code_to_index, axis=1)

In [ ]:
df['event_vector'].head()

In [ ]:
df['event_vector'].size

In [ ]:
df['actor_vector'].size

In [ ]:
df['time'].size

In [ ]:
# Convert the list of vectors to a matrix
df['combined_vector'] = df.apply(lambda row: np.concatenate([[row['time']] ,row['actor_vector'], row['event_vector']]), axis=1)

In [ ]:
df['combined_vector'].head()

In [ ]:
df_combined=df['combined_vector'].copy()

In [ ]:
del df

In [ ]:
# Define column names for actor_vector and event_vector
actor_columns = [f'actor_{actor}' for actor in important_actors]
event_columns = [f'event_{code}' for code in all_unique_codes]

# Combine the column names
combined_columns = ['time']+actor_columns + event_columns
# Convert the numpy array to a DataFrame
combined_df = pd.DataFrame(data=np.vstack(df_combined.values), columns=combined_columns)

In [ ]:
combined_df.head()

In [ ]:
del df_combined

In [ ]:
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client()

# Define the table ID
table_id = "factoreddatathon2014.GDELT.training_set_kmeans"

# Chunk size
chunk_size = 1000000  # Adjust based on your memory limits

# Upload in chunks
for i in range(0, len(combined_df), chunk_size):
    chunk = combined_df.iloc[i:i + chunk_size]
    job = client.load_table_from_dataframe(chunk, table_id)
    job.result()  # Wait for the job to complete

    print(f"Loaded chunk {i // chunk_size + 1} into {table_id}")
